In [9]:
import os
import pandas as pd
import numpy as np
import os.path as op
import nibabel as nb
from nilearn.glm.first_level import FirstLevelModel
from nilearn.plotting import plot_stat_map
from itertools import combinations  # For computing contrasts between conditions

# Define the function to adjust the event timetable based on motion information
def motion_controlled_event_timetable(event_table, fd_data, six_absolute_motion, TR, FD_thr, ab_motion_thr):
    # Detect timepoints where FD exceeds the threshold
    out_motion_detect = fd_data.to_numpy()[:] > FD_thr
    out_motion_index = np.where(out_motion_detect == True)
    # Detect timepoints where any of the six motion parameters exceed the absolute motion threshold
    six_motion_ex = np.where(np.sum((six_absolute_motion > ab_motion_thr) == True, 1) > 0)
    
    # Convert motion timepoints to actual time by multiplying with TR
    out_motion_time = np.array([])
    if len(out_motion_index[0]) > 0:
        out_motion_time = (out_motion_index[0][:] + 1) * TR
    if len(six_motion_ex[0]) > 0:
        six_motion_time = (six_motion_ex[0] + 1) * TR
        out_motion_time = np.concatenate((out_motion_time, six_motion_time), axis=0)
        out_motion_time = np.unique(out_motion_time)
    
    tmp_timetable = event_table.assign(time_end=lambda dataframe: dataframe['onset'] + dataframe['duration'])
    tmp_timetable = tmp_timetable.reset_index(drop=True)
    
    # Mark the timepoints where motion exceeds thresholds
    block_time_judge = np.zeros(tmp_timetable.shape[0])
    block_time_in = np.zeros(tmp_timetable.shape[0])
    try:
        for n_time in range(tmp_timetable.shape[0]):
            for i in out_motion_time:
                time_judge_0 = (i <= tmp_timetable.loc[n_time, 'time_end'])
                block_time_judge[n_time] += time_judge_0
                time_judge_1 = (i <= tmp_timetable.loc[n_time, 'time_end']) * (i >= tmp_timetable.loc[n_time, 'onset'])
                block_time_in[n_time] += time_judge_1
            
        tmp_timetable = tmp_timetable.assign(
            time_delete=block_time_judge * TR,
            delete_time_inblock=block_time_in
        )
        tmp_timetable.loc[:, 'duration'] = tmp_timetable['duration'] - tmp_timetable['delete_time_inblock'] * TR
        
        # Adjust onset times and recalculate time_end
        for n_time in range(tmp_timetable.shape[0]):
            if n_time != 0:
                tmp_timetable.loc[n_time, 'onset'] = tmp_timetable.loc[n_time, 'onset'] - tmp_timetable.loc[n_time, 'time_delete']
            tmp_timetable.loc[n_time, 'time_end'] = tmp_timetable.loc[n_time, 'onset'] + tmp_timetable.loc[n_time, 'duration']
    except Exception as e:
        print("Error in motion_controlled_event_timetable:", e)
        out_motion_time = False
        tmp_timetable = event_table
    return [tmp_timetable, out_motion_time]

# Define the function to correct NIfTI data based on motion information
def correct_motion_for_niidata(motion_corrected_path, subname, run, task_file, nii_data, TR, out_motion_time):
    motion_corrected_subfolder = op.join(motion_corrected_path, subname)
    if not os.path.exists(motion_corrected_subfolder):
        os.makedirs(motion_corrected_subfolder)
    motion_corrected_nii = op.join(motion_corrected_subfolder, subname + task_file)
    niidata = nb.load(nii_data)
    timepoints_to_delete = ((out_motion_time / TR).astype('int64')) - 1
    motion_corrected_data = np.delete(niidata.get_fdata(), timepoints_to_delete, axis=3)
    motion_corrected_nii_data = nb.Nifti1Image(motion_corrected_data, header=niidata.header, affine=niidata.affine)
    motion_corrected_nii_data.header.set_data_dtype(np.int16)
    nb.save(motion_corrected_nii_data, motion_corrected_nii)
    return motion_corrected_nii

roodir = '/mnt/d/language_atlas_project/newdata/ds001734'
sublist = ['001']  # Add more subjects as needed
runs = ['01', '02', '03', '04']
taskname = 'MGT'
TR = 1.0  # Adjust TR based on your data
FD_thr = 0.2  # Framewise displacement threshold
ab_motion_thr = 3  # Absolute motion threshold

# Output paths
out_path = op.join(roodir, 'derivatives', 'first_level_model_corrected_nii')
motion_corrected_path = op.join(roodir, 'derivatives', 'motion_corrected_data_nii')
# Set whether to perform motion exclusion
do_motion_exclusion = False  # Set to False to avoid deleting frames due to motion
for sub in sublist:
    subname = 'sub-' + sub
    subeventdir = op.join(roodir, subname, 'func')
    subimagedir = op.join(roodir, 'derivatives', 'fmriprep', subname, 'func')
    for run in runs:
        sub_event_file = op.join(subeventdir, f'{subname}_task-{taskname}_run-{run}_events.tsv')
        sub_niidata_file = op.join(subimagedir, f'{subname}_task-{taskname}_run-{run}_bold_space-MNI152NLin2009cAsym_preproc.nii.gz')
        sub_motioninfo_file = op.join(subimagedir, f'{subname}_task-{taskname}_run-{run}_bold_confounds.tsv')

        # Check if all necessary files exist
        if not (os.path.exists(sub_event_file) and os.path.exists(sub_niidata_file) and os.path.exists(sub_motioninfo_file)):
            print(f"Missing files for {subname}, run {run}")
            continue

        # Load event data
        event_data = pd.read_csv(sub_event_file, sep='\t')
        # Rename 'participant_response' to 'trial_type'
        if 'participant_response' in event_data.columns:
            event_data.rename(columns={'participant_response': 'trial_type'}, inplace=True)
        else:
            print(f"'participant_response' column not found in {sub_event_file}")
            continue

        # Load motion parameters data
        confounds = pd.read_csv(sub_motioninfo_file, sep='\t')
        # Handle missing values
        confounds = confounds.fillna(0)

        # Get framewise displacement (FD) data
        if 'FramewiseDisplacement' in confounds.columns:
            fd = confounds[['FramewiseDisplacement']]
        elif 'framewise_displacement' in confounds.columns:
            fd = confounds[['framewise_displacement']]
        else:
            print(f"FD column not found in {sub_motioninfo_file}")
            continue

        # Get six motion parameters
        motion_params = confounds[['X', 'Y', 'Z', 'RotX', 'RotY', 'RotZ']]

        # Motion correction
        if do_motion_exclusion:
            [event_data_corrected, out_motion_time] = motion_controlled_event_timetable(event_data, fd, motion_params, TR, FD_thr, ab_motion_thr)
            # Save corrected event data
            out_sub_path = op.join(out_path, subname)
            if not os.path.exists(out_sub_path):
                os.makedirs(out_sub_path)
            event_out_file = op.join(out_sub_path, f'{subname}_task-{taskname}_run-{run}_events_corrected.tsv')
            event_data_corrected.to_csv(event_out_file, sep='\t', index=False)
            # Correct NIfTI data
            if not isinstance(out_motion_time, bool):
                task_file = f'_task-{taskname}_run-{run}_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'
                corrected_nii_file = correct_motion_for_niidata(motion_corrected_path, subname, run, task_file, sub_niidata_file, TR, out_motion_time)
                fmri_img = nb.load(corrected_nii_file)
                # Adjust motion parameters
                timepoints_to_delete = ((out_motion_time / TR).astype('int64')) - 1
                motion_params_corrected = motion_params.drop(motion_params.index[timepoints_to_delete]).reset_index(drop=True)
            else:
                fmri_img = nb.load(sub_niidata_file)
                motion_params_corrected = motion_params
                event_data_corrected = event_data
        else:
            event_data_corrected = event_data
            fmri_img = nb.load(sub_niidata_file)
            motion_params_corrected = motion_params

        # Perform first-level GLM analysis
        fmri_glm = FirstLevelModel(
            t_r=TR,
            noise_model='ar1',
            hrf_model='spm',
            drift_model=None,
            high_pass=1./128,  # Adjust the high-pass filter as needed
            signal_scaling=False,
            minimize_memory=False
        )

        fmri_glm = fmri_glm.fit(
            fmri_img,
            events=event_data_corrected,
            confounds=motion_params_corrected
        )

        # Define contrasts
        # Assuming 'trial_type' column contains condition names
        conditions = event_data_corrected['trial_type'].unique()
        design_matrix = fmri_glm.design_matrices_[0]

        # Create contrasts for each condition
        contrasts = {}
        for cond in conditions:
            # The columns corresponding to the condition
            cond_vector = np.array([1 if c == cond else 0 for c in design_matrix.columns])
            contrasts[cond] = cond_vector
        # Compute contrasts for each condition vs baseline
        out_sub_path = op.join(out_path, subname)
        stats_results_path = op.join(out_sub_path, 'stats_results', f'run-{run}')
        if not os.path.exists(stats_results_path):
            os.makedirs(stats_results_path)
        for contrast_id, contrast_val in contrasts.items():
            z_map = fmri_glm.compute_contrast(contrast_val, output_type='z_score')
            z_map_file = op.join(stats_results_path, f'{subname}_task-{taskname}_run-{run}_{contrast_id}_zmap.nii.gz')
            z_map.to_filename(z_map_file)
            print(f"Contrast {contrast_id} for {subname}, run {run} completed and saved to {z_map_file}")
            # Optionally, plot the contrast map
            # plot_stat_map(z_map, title=f'{subname} {contrast_id}', display_mode='ortho', threshold=3.0)
        # Compute pairwise differences between conditions
        for cond1, cond2 in combinations(conditions, 2):
            contrast_vector = contrasts[cond1] - contrasts[cond2]
            contrast_id = f'{cond1} - {cond2}'
            z_map = fmri_glm.compute_contrast(contrast_vector, output_type='z_score')
            z_map_file = op.join(stats_results_path, f'{subname}_task-{taskname}_run-{run}_{contrast_id}_zmap.nii.gz')
            z_map.to_filename(z_map_file)
            print(f"Contrast {contrast_id} for {subname}, run {run} completed and saved to {z_map_file}")

            # Optionally, plot the contrast map
            # plot_stat_map(z_map, title=f'{subname} {contrast_id}', display_mode='ortho', threshold=3.0)

/home/wuguowei/anaconda3/envs/nipype/lib/python3.10/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: gain, RT, loss
  warnings.warn(


Contrast weakly_accept for sub-001, run 01 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_nii/sub-001/stats_results/run-01/sub-001_task-MGT_run-01_weakly_accept_zmap.nii.gz
Contrast strongly_accept for sub-001, run 01 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_nii/sub-001/stats_results/run-01/sub-001_task-MGT_run-01_strongly_accept_zmap.nii.gz
Contrast weakly_reject for sub-001, run 01 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_nii/sub-001/stats_results/run-01/sub-001_task-MGT_run-01_weakly_reject_zmap.nii.gz
Contrast strongly_reject for sub-001, run 01 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_nii/sub-001/stats_results/run-01/sub-001_task-MGT_run-01_strongly_reject_zmap.nii.gz
Contrast NoResp for sub-001, run 01 completed an

/home/wuguowei/anaconda3/envs/nipype/lib/python3.10/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: gain, RT, loss
  warnings.warn(


Contrast strongly_accept for sub-001, run 02 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_nii/sub-001/stats_results/run-02/sub-001_task-MGT_run-02_strongly_accept_zmap.nii.gz
Contrast weakly_accept for sub-001, run 02 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_nii/sub-001/stats_results/run-02/sub-001_task-MGT_run-02_weakly_accept_zmap.nii.gz
Contrast strongly_reject for sub-001, run 02 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_nii/sub-001/stats_results/run-02/sub-001_task-MGT_run-02_strongly_reject_zmap.nii.gz
Contrast weakly_reject for sub-001, run 02 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_nii/sub-001/stats_results/run-02/sub-001_task-MGT_run-02_weakly_reject_zmap.nii.gz
Contrast strongly_accept - weakly_accept for sub

/home/wuguowei/anaconda3/envs/nipype/lib/python3.10/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: gain, RT, loss
  warnings.warn(


Contrast weakly_accept for sub-001, run 03 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_nii/sub-001/stats_results/run-03/sub-001_task-MGT_run-03_weakly_accept_zmap.nii.gz
Contrast strongly_accept for sub-001, run 03 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_nii/sub-001/stats_results/run-03/sub-001_task-MGT_run-03_strongly_accept_zmap.nii.gz
Contrast weakly_reject for sub-001, run 03 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_nii/sub-001/stats_results/run-03/sub-001_task-MGT_run-03_weakly_reject_zmap.nii.gz
Contrast strongly_reject for sub-001, run 03 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_nii/sub-001/stats_results/run-03/sub-001_task-MGT_run-03_strongly_reject_zmap.nii.gz
Contrast weakly_accept - strongly_accept for sub

/home/wuguowei/anaconda3/envs/nipype/lib/python3.10/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: gain, RT, loss
  warnings.warn(


Contrast weakly_accept for sub-001, run 04 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_nii/sub-001/stats_results/run-04/sub-001_task-MGT_run-04_weakly_accept_zmap.nii.gz
Contrast strongly_accept for sub-001, run 04 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_nii/sub-001/stats_results/run-04/sub-001_task-MGT_run-04_strongly_accept_zmap.nii.gz
Contrast strongly_reject for sub-001, run 04 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_nii/sub-001/stats_results/run-04/sub-001_task-MGT_run-04_strongly_reject_zmap.nii.gz
Contrast weakly_reject for sub-001, run 04 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_nii/sub-001/stats_results/run-04/sub-001_task-MGT_run-04_weakly_reject_zmap.nii.gz
Contrast weakly_accept - strongly_accept for sub

## 接下来处理皮层数据的激活结果

In [14]:
import os
import pandas as pd
import numpy as np
import os.path as op
import sys
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import nibabel as nb
from nilearn.glm.first_level import make_first_level_design_matrix, run_glm
from nilearn.glm.contrasts import compute_contrast
from itertools import combinations  # For computing contrasts between conditions

# Define the function to adjust the event timetable based on motion information
def motion_controlled_event_timetable(event_table, fd_data, six_absolute_motion, TR, FD_thr, ab_motion_thr):
    # Detect timepoints where FD exceeds the threshold
    out_motion_detect = fd_data.to_numpy().flatten() > FD_thr
    out_motion_index = np.where(out_motion_detect)[0]
    # Detect timepoints where any of the six motion parameters exceed the absolute motion threshold
    six_motion_ex = np.where(np.any(np.abs(six_absolute_motion) > ab_motion_thr, axis=1))[0]
    
    # Convert motion timepoints to actual time by multiplying with TR
    out_motion_time = np.array([])
    if len(out_motion_index) > 0:
        out_motion_time = (out_motion_index + 1) * TR
    if len(six_motion_ex) > 0:
        six_motion_time = (six_motion_ex + 1) * TR
        out_motion_time = np.concatenate((out_motion_time, six_motion_time), axis=0)
        out_motion_time = np.unique(out_motion_time)
    
    tmp_timetable = event_table.assign(time_end=lambda dataframe: dataframe['onset'] + dataframe['duration'])
    tmp_timetable = tmp_timetable.reset_index(drop=True)
    
    # Mark the timepoints where motion exceeds thresholds
    block_time_judge = np.zeros(tmp_timetable.shape[0])
    block_time_in = np.zeros(tmp_timetable.shape[0])
    try:
        for n_time in range(tmp_timetable.shape[0]):
            for i in out_motion_time:
                time_judge_0 = (i <= tmp_timetable.loc[n_time, 'time_end'])
                block_time_judge[n_time] += time_judge_0
                time_judge_1 = (i <= tmp_timetable.loc[n_time, 'time_end']) and (i >= tmp_timetable.loc[n_time, 'onset'])
                block_time_in[n_time] += time_judge_1
            
        tmp_timetable = tmp_timetable.assign(
            time_delete=block_time_judge * TR,
            delete_time_inblock=block_time_in
        )
        tmp_timetable.loc[:, 'duration'] = tmp_timetable['duration'] - tmp_timetable['delete_time_inblock'] * TR
        
        # Adjust onset times and recalculate time_end
        for n_time in range(tmp_timetable.shape[0]):
            if n_time != 0:
                tmp_timetable.loc[n_time, 'onset'] = tmp_timetable.loc[n_time, 'onset'] - tmp_timetable.loc[n_time, 'time_delete']
            tmp_timetable.loc[n_time, 'time_end'] = tmp_timetable.loc[n_time, 'onset'] + tmp_timetable.loc[n_time, 'duration']
    except Exception as e:
        print("Error in motion_controlled_event_timetable:", e)
        out_motion_time = False
        tmp_timetable = event_table
    return [tmp_timetable, out_motion_time]

# Define the function to correct motion in GIFTI data
def correct_motion_for_giidata(motion_corrected_path, subname, run, task_file_L, task_file_R, data_L_path, data_R_path, TR, out_motion_time):
    motion_corrected_subfolder = op.join(motion_corrected_path, subname)
    if not os.path.exists(motion_corrected_subfolder):
        os.makedirs(motion_corrected_subfolder)
    corrected_gii_file_L = op.join(motion_corrected_subfolder, subname + task_file_L)
    corrected_gii_file_R = op.join(motion_corrected_subfolder, subname + task_file_R)
    
    # Load GIFTI data
    data_L = nb.load(data_L_path)
    data_R = nb.load(data_R_path)
    
    # Calculate timepoints to delete
    timepoints_to_delete = ((out_motion_time / TR).astype(int)) - 1
    timepoints_to_keep = np.setdiff1d(np.arange(len(data_L.darrays)), timepoints_to_delete)
    
    # Create new GIFTI images with selected timepoints
    corrected_darrays_L = [data_L.darrays[i] for i in timepoints_to_keep]
    corrected_darrays_R = [data_R.darrays[i] for i in timepoints_to_keep]
    
    corrected_data_L = nb.gifti.GiftiImage(darrays=corrected_darrays_L)
    corrected_data_R = nb.gifti.GiftiImage(darrays=corrected_darrays_R)
    
    # Save corrected GIFTI data
    nb.save(corrected_data_L, corrected_gii_file_L)
    nb.save(corrected_data_R, corrected_gii_file_R)
    
    return corrected_gii_file_L, corrected_gii_file_R

# Main code
roodir = '/mnt/d/language_atlas_project/newdata/ds001734'
sublist = ['001']  # Add more subjects as needed
runs = ['01', '02', '03', '04']
taskname = 'MGT'
TR = 1.0  # Adjust TR based on your data
FD_thr = 0.2  # Framewise displacement threshold
ab_motion_thr = 3  # Absolute motion threshold

# Output paths
out_path = op.join(roodir, 'derivatives', 'first_level_model_corrected_gii')
motion_corrected_path = op.join(roodir, 'derivatives', 'motion_corrected_data_gii')

# Set whether to perform motion exclusion
do_motion_exclusion = False  # Set to False to avoid deleting frames due to motion

for sub in sublist:
    subname = 'sub-' + sub
    subeventdir = op.join(roodir, subname, 'func')
    subimagedir = op.join(roodir, 'derivatives', 'fmriprep', subname, 'func')
    for run in runs:
        sub_event_file = op.join(subeventdir, f'{subname}_task-{taskname}_run-{run}_events.tsv')
        sub_gii_file_L = op.join(subimagedir, f'{subname}_task-{taskname}_run-{run}_bold_space-fsaverage5.L.func.gii')
        sub_gii_file_R = op.join(subimagedir, f'{subname}_task-{taskname}_run-{run}_bold_space-fsaverage5.R.func.gii')
        sub_motioninfo_file = op.join(subimagedir, f'{subname}_task-{taskname}_run-{run}_bold_confounds.tsv')

        # Check if all necessary files exist
        if not (os.path.exists(sub_event_file) and os.path.exists(sub_gii_file_L) and os.path.exists(sub_gii_file_R) and os.path.exists(sub_motioninfo_file)):
            print(f"Missing files for {subname}, run {run}")
            continue

        # Load event data
        event_data = pd.read_csv(sub_event_file, sep='\t')
        # Rename 'participant_response' to 'trial_type'
        if 'participant_response' in event_data.columns:
            event_data.rename(columns={'participant_response': 'trial_type'}, inplace=True)
        else:
            print(f"'participant_response' column not found in {sub_event_file}")
            continue

        # Load motion parameters data
        confounds = pd.read_csv(sub_motioninfo_file, sep='\t')
        # Handle missing values
        confounds = confounds.fillna(0)

        # Get framewise displacement (FD) data
        if 'FramewiseDisplacement' in confounds.columns:
            fd = confounds[['FramewiseDisplacement']]
        elif 'framewise_displacement' in confounds.columns:
            fd = confounds[['framewise_displacement']]
        else:
            print(f"FD column not found in {sub_motioninfo_file}")
            continue

        # Get six motion parameters
        motion_params = confounds[['X', 'Y', 'Z', 'RotX', 'RotY', 'RotZ']]

        # Motion correction
        if do_motion_exclusion:
            # [Motion correction code remains unchanged]
            # ...
            pass
        else:
            event_data_corrected = event_data
            data_L = nb.load(sub_gii_file_L)
            data_R = nb.load(sub_gii_file_R)
            motion_params_corrected = motion_params

        # Concatenate left and right hemisphere data
        n_timepoints = len(data_L.darrays)
        n_vertices_L = data_L.darrays[0].data.shape[0]
        n_vertices_R = data_R.darrays[0].data.shape[0]

        # Initialize data matrix
        data_matrix = np.zeros((n_timepoints, n_vertices_L + n_vertices_R))
        for t in range(n_timepoints):
            data_L_t = data_L.darrays[t].data
            data_R_t = data_R.darrays[t].data
            data_matrix[t, :n_vertices_L] = data_L_t
            data_matrix[t, n_vertices_L:] = data_R_t

        # Build design matrix
        frame_times = TR * (np.arange(n_timepoints))
        design_matrix = make_first_level_design_matrix(
            frame_times,
            event_data_corrected,
            drift_model='polynomial',
            drift_order=3,
            add_regs=motion_params_corrected,
            add_reg_names=motion_params_corrected.columns,
            hrf_model='spm'
        )

        # Perform GLM analysis
        # Do NOT transpose data_matrix; Y should have shape (n_samples, n_voxels)
        labels, estimates = run_glm(data_matrix, design_matrix.values)

        # Define contrasts
        conditions = event_data_corrected['trial_type'].unique()
        design_columns = design_matrix.columns

        # Create contrasts for each condition
        contrasts = {}
        for cond in conditions:
            # The columns corresponding to the condition
            cond_vector = np.array([1 if cond == col else 0 for col in design_columns])
            contrasts[cond] = cond_vector

        # Prepare output directories
        out_sub_path = op.join(out_path, subname)
        stats_results_path = op.join(out_sub_path, 'stats_results', f'run-{run}')
        if not os.path.exists(stats_results_path):
            os.makedirs(stats_results_path)

        # Compute contrasts for each condition vs baseline
        for contrast_id, contrast_val in contrasts.items():
            contrast = compute_contrast(labels, estimates, contrast_val)
            # Compute Z-map
            z_map = contrast.z_score()
            # z_map has shape (n_voxels,)
            # Split z_map back to left and right hemispheres
            z_map_L = z_map[:n_vertices_L]
            z_map_R = z_map[n_vertices_L:]

            # Create GIFTI DataArrays
            z_map_L_darray = nb.gifti.GiftiDataArray(data=np.int32(z_map_L))
            z_map_R_darray = nb.gifti.GiftiDataArray(data=np.int32(z_map_R))

            # Create GIFTI images
            z_map_img_L = nb.gifti.GiftiImage(darrays=[z_map_L_darray])
            z_map_img_R = nb.gifti.GiftiImage(darrays=[z_map_R_darray])

            # Save GIFTI images
            z_map_file_L = op.join(stats_results_path, f'{subname}_task-{taskname}_run-{run}_{contrast_id}_zmap.L.func.gii')
            z_map_file_R = op.join(stats_results_path, f'{subname}_task-{taskname}_run-{run}_{contrast_id}_zmap.R.func.gii')
            nb.save(z_map_img_L, z_map_file_L)
            nb.save(z_map_img_R, z_map_file_R)

            print(f"Contrast {contrast_id} for {subname}, run {run} completed and saved to {z_map_file_L} and {z_map_file_R}")

        # Compute pairwise differences between conditions
        for cond1, cond2 in combinations(conditions, 2):
            contrast_vector = contrasts[cond1] - contrasts[cond2]
            contrast_id = f'{cond1} - {cond2}'

            contrast = compute_contrast(labels, estimates, contrast_vector)
            # Compute Z-map
            z_map = contrast.z_score()
            # z_map has shape (n_voxels,)
            # Split z_map back to left and right hemispheres
            z_map_L = z_map[:n_vertices_L]
            z_map_R = z_map[n_vertices_L:]

            # Create GIFTI DataArrays
            z_map_L_darray = nb.gifti.GiftiDataArray(data=np.int32(z_map_L))
            z_map_R_darray = nb.gifti.GiftiDataArray(data=np.int32(z_map_R))

            # Create GIFTI images
            z_map_img_L = nb.gifti.GiftiImage(darrays=[z_map_L_darray])
            z_map_img_R = nb.gifti.GiftiImage(darrays=[z_map_R_darray])

            # Save GIFTI images
            z_map_file_L = op.join(stats_results_path, f'{subname}_task-{taskname}_run-{run}_{contrast_id}_zmap.L.func.gii')
            z_map_file_R = op.join(stats_results_path, f'{subname}_task-{taskname}_run-{run}_{contrast_id}_zmap.R.func.gii')
            nb.save(z_map_img_L, z_map_file_L)
            nb.save(z_map_img_R, z_map_file_R)

            print(f"Contrast {contrast_id} for {subname}, run {run} completed and saved to {z_map_file_L} and {z_map_file_R}")

/home/wuguowei/anaconda3/envs/nipype/lib/python3.10/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: gain, RT, loss
  warnings.warn(
/home/wuguowei/anaconda3/envs/nipype/lib/python3.10/site-packages/nilearn/glm/_utils.py:205: RuntimeWarning: divide by zero encountered in divide
  return np.where(X <= 0, 0, 1.0 / X)


Contrast weakly_accept for sub-001, run 01 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_gii/sub-001/stats_results/run-01/sub-001_task-MGT_run-01_weakly_accept_zmap.L.func.gii and /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_gii/sub-001/stats_results/run-01/sub-001_task-MGT_run-01_weakly_accept_zmap.R.func.gii
Contrast strongly_accept for sub-001, run 01 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_gii/sub-001/stats_results/run-01/sub-001_task-MGT_run-01_strongly_accept_zmap.L.func.gii and /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_gii/sub-001/stats_results/run-01/sub-001_task-MGT_run-01_strongly_accept_zmap.R.func.gii
Contrast weakly_reject for sub-001, run 01 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_gii/s

/home/wuguowei/anaconda3/envs/nipype/lib/python3.10/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: gain, RT, loss
  warnings.warn(
/home/wuguowei/anaconda3/envs/nipype/lib/python3.10/site-packages/nilearn/glm/_utils.py:205: RuntimeWarning: divide by zero encountered in divide
  return np.where(X <= 0, 0, 1.0 / X)


Contrast strongly_accept for sub-001, run 02 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_gii/sub-001/stats_results/run-02/sub-001_task-MGT_run-02_strongly_accept_zmap.L.func.gii and /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_gii/sub-001/stats_results/run-02/sub-001_task-MGT_run-02_strongly_accept_zmap.R.func.gii
Contrast weakly_accept for sub-001, run 02 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_gii/sub-001/stats_results/run-02/sub-001_task-MGT_run-02_weakly_accept_zmap.L.func.gii and /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_gii/sub-001/stats_results/run-02/sub-001_task-MGT_run-02_weakly_accept_zmap.R.func.gii
Contrast strongly_reject for sub-001, run 02 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_gii

/home/wuguowei/anaconda3/envs/nipype/lib/python3.10/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: gain, RT, loss
  warnings.warn(
/home/wuguowei/anaconda3/envs/nipype/lib/python3.10/site-packages/nilearn/glm/_utils.py:205: RuntimeWarning: divide by zero encountered in divide
  return np.where(X <= 0, 0, 1.0 / X)


Contrast weakly_accept for sub-001, run 03 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_gii/sub-001/stats_results/run-03/sub-001_task-MGT_run-03_weakly_accept_zmap.L.func.gii and /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_gii/sub-001/stats_results/run-03/sub-001_task-MGT_run-03_weakly_accept_zmap.R.func.gii
Contrast strongly_accept for sub-001, run 03 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_gii/sub-001/stats_results/run-03/sub-001_task-MGT_run-03_strongly_accept_zmap.L.func.gii and /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_gii/sub-001/stats_results/run-03/sub-001_task-MGT_run-03_strongly_accept_zmap.R.func.gii
Contrast weakly_reject for sub-001, run 03 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_gii/s

/home/wuguowei/anaconda3/envs/nipype/lib/python3.10/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: gain, RT, loss
  warnings.warn(
/home/wuguowei/anaconda3/envs/nipype/lib/python3.10/site-packages/nilearn/glm/_utils.py:205: RuntimeWarning: divide by zero encountered in divide
  return np.where(X <= 0, 0, 1.0 / X)


Contrast weakly_accept for sub-001, run 04 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_gii/sub-001/stats_results/run-04/sub-001_task-MGT_run-04_weakly_accept_zmap.L.func.gii and /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_gii/sub-001/stats_results/run-04/sub-001_task-MGT_run-04_weakly_accept_zmap.R.func.gii
Contrast strongly_accept for sub-001, run 04 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_gii/sub-001/stats_results/run-04/sub-001_task-MGT_run-04_strongly_accept_zmap.L.func.gii and /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_gii/sub-001/stats_results/run-04/sub-001_task-MGT_run-04_strongly_accept_zmap.R.func.gii
Contrast strongly_reject for sub-001, run 04 completed and saved to /mnt/d/language_atlas_project/newdata/ds001734/derivatives/first_level_model_corrected_gii